In [2]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## モデルをチェインとして記述する

In [10]:
l1 = L.Linear(4,3)
l2 = L.Linear(3,2)

def my_forward(x):
    h = l1(x)
    return l2(h)

#4要素のベクトルの2個のミニバッチ
data = np.array([[1,2,3,4],
                [4,5,6,7]], dtype=np.float32)
x = Variable(data)

#forward計算で2要素のベクトル×2が出て来る
my_forward(x).data


array([[-3.09469151,  0.32012695],
       [-5.90927601,  0.93085921]], dtype=float32)

一応関数で此のようにモデルを書くことはできるが、再利用が難しい

クラスで書くほうが望ましい


In [11]:
class MyProc(object):
    def __init__(self):
        self.l1 = L.Linear(4,3)
        self.l2 = L.Linear(3,2)
    
    def forward(self, x):
        h = self.l1(x)
        return self.l2(h)

クラスで書くとこのようになる
さらに、CPU/GPUサポートや、save/load機能などをサポートするにはChainクラスを継承することで可能となる

In [12]:
class MyChain(Chain):
    def __init__(self):
        self.l1 = L.Linear(4,3)
        self.l2 = L.Linear(3,2)
    
    def forward(self, x):
        h = self.l1(x)
        return self.l2(h)

このように書いた時、このMyChainの中のl1やl2のリンクを、MyChainの子リンクと呼んだりする

さらに、ChainクラスはLinkを継承している。これにより、さらに複雑なチェインがMyChainを子リンクとして持つことも可能

In [15]:
class MyChain(Chain):
    def __init__(self):
        self.l1 = L.Linear(4,3)
        self.l2 = L.Linear(3,2)
    
    def __call__(self, x):
        h = self.l1(x)
        return self.l2(h)

#インスタンス作成
ch = MyChain()
#__call__の定義により関数のように使うことができる
ch(x).data


array([[-1.73737729, -1.29845977],
       [-2.96359181, -2.7765975 ]], dtype=float32)

クラス内で__call__を定義すると関数のように呼ぶことができる

### ChainList

In [ ]:
class MyChain2(ChainList):
    def __init__(self):
        super(MyChain2, self).__init__(
            L.Linear(4,3),
            L.Linear(3,2)
        )
        
    def __call__(self, x):
        h = self[0](x)
        return self[1](h)

ChainListを継承すると任意の数のリンクを便利に扱うことができる

ただしリンク数が固定の場合はChainクラスを継承することが推奨される